Harjoitustyön aiheena on ennustaa kuluttajan anatama arvosana Airbnb kohteelle

Importataan ja asennetaan kirjastoja, joita voidaan hyödyntää datan käsittelyssä

In [7]:
!pip install sklearn
!pip install numpy

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1309 sha256=8190085c97bec69ec16bb41b6ccbaba8c2bc06eef58984dd77024f49d6a39729
  Stored in directory: c:\users\teemu\appdata\local\pip\cache\wheels\e4\7b\98\b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully built sklearn


In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer as Imputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.svm import SVC

from warnings import filterwarnings
filterwarnings('ignore')

In [17]:
df = pd.read_csv("Desktop/listings.csv")
df = df[:10000]
df.head()

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,18674,https://www.airbnb.com/rooms/18674,20211207182400,2021-12-08,Huge flat for 8 people close to Sagrada Familia,110m2 apartment to rent in Barcelona. Located ...,Apartment in Barcelona located in the heart of...,https://a0.muscache.com/pictures/13031453/413c...,71615,https://www.airbnb.com/users/show/71615,...,4.90,4.71,4.29,HUTB-002062,t,19,19,0,0,0.21
1,23197,https://www.airbnb.com/rooms/23197,20211207182400,2021-12-08,Forum CCIB DeLuxe★Spacious &Elegant★Large Balcony,"Beautiful spacious apartment, large terrace, 5...","Strategically located in the Parc del Fòrum, a...",https://a0.muscache.com/pictures/miso/Hosting-...,90417,https://www.airbnb.com/users/show/90417,...,4.98,4.66,4.67,HUTB-005057,f,2,2,0,0,0.41
2,32711,https://www.airbnb.com/rooms/32711,20211207182400,2021-12-08,Sagrada Familia area - Còrsega 1,A lovely two bedroom apartment only 250 m from...,What's nearby <br />This apartment is located...,https://a0.muscache.com/pictures/357b25e4-f414...,135703,https://www.airbnb.com/users/show/135703,...,4.79,4.81,4.40,HUTB-001722,t,3,3,0,0,0.50
3,34981,https://www.airbnb.com/rooms/34981,20211207182400,2021-12-07,VIDRE HOME PLAZA REAL on LAS RAMBLAS,Spacious apartment for large families or group...,"Located in Ciutat Vella in the Gothic Quarter,...",https://a0.muscache.com/pictures/c4d1723c-e479...,73163,https://www.airbnb.com/users/show/73163,...,4.68,4.73,4.47,HUTB-001506,f,2,2,0,0,1.17
4,35318,https://www.airbnb.com/rooms/35318,20211207182400,2021-12-07,Luxury room with private bathroom and balcony,"Luxury Room with King Size bed, private bathro...",The Gothic Quarter of Barcelona. One of the ol...,https://a0.muscache.com/pictures/miso/Hosting-...,152070,https://www.airbnb.com/users/show/152070,...,4.83,4.81,4.68,NaN,t,1,0,1,0,1.89


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 74 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            10000 non-null  int64  
 1   listing_url                                   10000 non-null  object 
 2   scrape_id                                     10000 non-null  int64  
 3   last_scraped                                  10000 non-null  object 
 4   name                                          9990 non-null   object 
 5   description                                   9961 non-null   object 
 6   neighborhood_overview                         6468 non-null   object 
 7   picture_url                                   10000 non-null  object 
 8   host_id                                       10000 non-null  int64  
 9   host_url                                      10000 non-null  

In [25]:
cols_to_remove = ['id','listing_url','scrape_id','last_scraped','name','description','neighborhood_overview','picture_url','host_id','host_url','host_name','host_about','host_thumbnail_url',
                  'host_picture_url','host_listings_count','neighbourhood','neighbourhood_cleansed','bathrooms','minimum_minimum_nights','maximum_minimum_nights','minimum_maximum_nights',
                  'maximum_maximum_nights','minimum_nights_avg_ntm','maximum_nights_avg_ntm','calendar_updated','calendar_last_scraped','first_review','last_review','license',]
data = df.drop(cols_to_remove, axis=1)

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 45 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   host_since                                    9973 non-null   object 
 1   host_location                                 9964 non-null   object 
 2   host_response_time                            6413 non-null   object 
 3   host_response_rate                            6413 non-null   object 
 4   host_acceptance_rate                          6575 non-null   object 
 5   host_is_superhost                             9973 non-null   object 
 6   host_neighbourhood                            7873 non-null   object 
 7   host_total_listings_count                     9973 non-null   float64
 8   host_verifications                            10000 non-null  object 
 9   host_has_profile_pic                          9973 non-null   